In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from keras.layers import Input, Average
from keras.models import Model
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [ ]:
pip install tensorflow==2.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: google-auth-oauthlib
    Found existing ins

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load pre-trained models with unique names
model_paths = {
    "ResNet50V2": "/content/drive/MyDrive/Final Models Emsemble/Main Models/RestNet50_model.h5",
    "ResNet101": "/content/drive/MyDrive/Final Models Emsemble/Main Models/ResNet101V2_model.h5",
    "DenseNet121": "/content/drive/MyDrive/Final Models Emsemble/Main Models/DenseNet121.h5",
    "DenseNet169": "/content/drive/MyDrive/Final Models Emsemble/Main Models/DenseNet169.h5",
    "XceptionNet": "/content/drive/MyDrive/Final Models Emsemble/Main Models/Xception_model.h5",
}

models = {}
for name, path in model_paths.items():
    model = tf.keras.models.load_model(path, compile=False)
    model._name = name  # Assign unique name to prevent conflicts
    model.trainable = False  # Freeze model to prevent training
    models[name] = model

# Define ensemble model
input_layer = tf.keras.layers.Input(shape=(224, 224, 3))
outputs = [model(input_layer) for model in models.values()]
average_output = tf.keras.layers.Average(name="Ensemble_Average")(outputs)
ensemble_model = tf.keras.models.Model(inputs=input_layer, outputs=average_output, name="EnsembleModel")

# Save and load ensemble model
ensemble_model.save('/content/drive/MyDrive/last/Ensemble_SoftVoting_Model.h5')

# Load the ensemble model with a different variable name
ensemble_model_loaded = tf.keras.models.load_model('/content/drive/MyDrive/last/Ensemble_SoftVoting_Model.h5')

# Print model summary
ensemble_model_loaded.summary()

# Set dataset paths
data_dir = "/content/drive/MyDrive/Dataset/NEW_DATASET/"
batch_size = 1  # Test images one by one

# Load test data
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    data_dir + "test",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Evaluate model
y_true, y_pred = [], []
for i in range(test_generator.samples):
    img, label = next(test_generator)  # Fetch batch
    pred = ensemble_model_loaded.predict(img, verbose=0)  # Use loaded ensemble model
    y_pred.append(np.argmax(pred, axis=1)[0])  # Get predicted class
    y_true.append(np.argmax(label, axis=1)[0])  # Get true class
    print(f"\rProcessing {i + 1}/{test_generator.samples}...", end="")

# Accuracy & classification report
accuracy = accuracy_score(y_true, y_pred)
print(f"\n🔥 Overall Model Accuracy: {accuracy * 100:.2f}%")
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "EnsembleModel"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 ResNet50V2 (Functional)     (None, 7)                    2413248   ['input_3[0][0]']             
                                                          7                                       
                                                                                                  
 ResNet101 (Functional)      (None, 7)                    4319424   ['input_3[0][0]']             
                                                          7                                       
                                                                                      